In [2]:
pip install tensorflow

     ------------------------------------ 385.0/385.0 MB 690.3 kB/s eta 0:00:00
  Using cached keras-3.5.0-py3-none-any.whl (1.1 MB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl (26.4 MB)
     -------------------------------------- 15.8/15.8 MB 948.3 kB/s eta 0:00:00
     ---------------------------------------- 4.3/4.3 MB 773.4 kB/s eta 0:00:00
  Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl (26 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
     ------------------------------------ 413.4/413.4 kB 662.0 kB/s eta 0:00:00
  Using cached termcolor-2.4.0-py3-none-any.whl (7.7 kB)
     ---------------------------------------- 1.5/1.5 MB 944.4 kB/s eta 0:00:00
     ---------------------------------------- 3.0/3.0 MB 744.9 kB/s eta 0:00:00
     ------------------------------------ 126.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.2.2 requires pyqt5<5.13, which is not installed.
spyder 5.2.2 requires pyqtwebengine<5.13, which is not installed.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.
scipy 1.9.3 requires numpy<1.26.0,>=1.18.5, but you have numpy 1.26.4 which is incompatible.
numba 0.56.3 requires numpy<1.24,>=1.18, but you have numpy 1.26.4 which is incompatible.


In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# Define sigmoid activation function
def sigmoid(x):
    return 1. / (1. + tf.exp(-x))

In [ ]:
# Define model function
def f(params, x):
    w0 = params[:10]
    b0 = params[10:20]
    w1 = params[20:30]
    b1 = params[30]
    x = sigmoid(x * w0 + b0)
    x = sigmoid(tf.reduce_sum(x * w1) + b1)
    return x

In [ ]:
# Initialize parameters
params = tf.Variable(np.random.normal(size=(31,)), dtype=tf.float32)

In [ ]:
# Compute the gradient of f with respect to x
def dfdx(params, x):
    with tf.GradientTape() as tape:
        tape.watch(x)
        result = f(params, x)
    return tape.gradient(result, x)


In [ ]:
# Prepare input values
inputs = np.linspace(-2., 2., num=401, dtype=np.float32)

In [ ]:
# Vectorize the function using map_fn in TensorFlow
f_vect = lambda params, x: tf.map_fn(lambda x_: f(params, x_), x)
dfdx_vect = lambda params, x: tf.map_fn(lambda x_: dfdx(params, x_), x)

In [ ]:
# Define the loss function
@tf.function
def loss(params, inputs):
    eq = dfdx_vect(params, inputs) +2*inputs*f_vect(params, inputs)
    #eq = dfdx_vect(params, inputs) - inputs - f_vect(params, inputs)
    ic = f(params, 0.) - 1.
    return tf.reduce_mean(eq**2) + ic**2

In [ ]:
# Compute the gradient of the loss function with respect to the parameters
@tf.function
def grad_loss(params, inputs):
    with tf.GradientTape() as tape:
        tape.watch(params)  # Ensure params are tracked
        loss_value = loss(params, inputs)
    return tape.gradient(loss_value, params)


In [ ]:
# Training loop
epochs = 10000
learning_rate = 0.01
momentum = 0.99
velocity = tf.Variable(np.zeros(31, dtype=np.float32))

for epoch in range(epochs):
    if epoch % 100 == 0:
        current_loss = loss(params, inputs).numpy()
        print(f'epoch: {epoch:3d} loss: {current_loss:.6f}')

    # Calculate gradients with respect to params
    gradient = grad_loss(params, inputs)

    # Ensure gradient is not None
    if gradient is not None:
        velocity.assign(momentum * velocity - learning_rate * gradient)
        params.assign_add(velocity)

In [ ]:
# Plot results
#exact_solution = 2 * np.exp(inputs) - inputs - 1
exact_solution =   np.exp(-inputs**2)
plt.plot(inputs, exact_solution, label='exact')
plt.plot(inputs, f_vect(params, inputs).numpy(), label='approx')
plt.legend()
plt.show()